In [1]:
import numpy as np
import tensorflow as tf

import tensorflow_datasets as tfds

C:\Users\pc\.conda\envs\TF\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
mnist_dataset, mnist_info = tfds.load(name='mnist', with_info=True, as_supervised=True)

## Preprocessing The MNIST Data

In [40]:
mnist_train, mnist_test = mnist_dataset['train'], mnist_dataset['test']
num_val = 0.1* mnist_info.splits['train'].num_examples
num_val = tf.cast(num_val, tf.int64)

num_test = 0.1* mnist_info.splits['test'].num_examples
num_test = tf.cast(num_test, tf.int64)

def scale(image, label):
    image = tf.cast(image, tf.float64)
    image /= 255.
    return image, label

scaled_train_val = mnist_train.map(scale)
scale_test = mnist_test.map(scale)
test_data = scale_test

buffer_size = 10000
shuffle_train_val = scaled_train_val.shuffle(buffer_size)

val_data = shuffle_train_val.take(num_val)
train_data = shuffle_train_val.skip(num_val)

batch_size = 100

train_data = train_data.batch(batch_size)
val_data = val_data.batch(num_val)
test_data = test_data.batch(num_test)

val_inputs, val_targets = next(iter(val_data))


## Outlining The Model

In [46]:
input_size = 784
output_size = 10
hidden_layer_size = 200

model = tf.keras.Sequential([
                             tf.keras.layers.Flatten(input_shape=(28,28,1)),
                             tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
                             tf.keras.layers.Dense(hidden_layer_size, activation='tanh'),
                             tf.keras.layers.Dense(output_size, activation='softmax')
                             ])

## Selecting The Optimizer And The Loss Function

In [47]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

## Training

In [48]:
num_epoch = 5
model.fit(train_data, epochs = num_epoch, validation_data = ( val_inputs, val_targets), verbose = 2)

Epoch 1/5
540/540 - 6s - loss: 0.2597 - accuracy: 0.9237 - val_loss: 0.1298 - val_accuracy: 0.9612 - 6s/epoch - 10ms/step
Epoch 2/5
540/540 - 4s - loss: 0.1011 - accuracy: 0.9688 - val_loss: 0.0871 - val_accuracy: 0.9737 - 4s/epoch - 7ms/step
Epoch 3/5
540/540 - 4s - loss: 0.0679 - accuracy: 0.9790 - val_loss: 0.0726 - val_accuracy: 0.9775 - 4s/epoch - 7ms/step
Epoch 4/5
540/540 - 4s - loss: 0.0489 - accuracy: 0.9842 - val_loss: 0.0579 - val_accuracy: 0.9835 - 4s/epoch - 7ms/step
Epoch 5/5
540/540 - 4s - loss: 0.0366 - accuracy: 0.9884 - val_loss: 0.0472 - val_accuracy: 0.9868 - 4s/epoch - 8ms/step


## Test the model

In [57]:
test_loss, test_accuracy = model.evaluate(test_data)
print(f"Test loss: {test_loss:.2f}. Test accuracy: {test_accuracy:.2f}")

10/10 [==============================] - 0s 31ms/step - loss: 0.0774 - accuracy: 0.9755
Test loss: 0.08. Test accuracy: 0.98
